## Feature Engineering

### Reading Data

In [18]:
import pandas as pd
import numpy as np

cleaned_df = pd.read_csv('../data/processed/cleaned_data.csv')
timeseries_df = cleaned_df.copy()
customer_metrics_df = cleaned_df.copy()

### Adding Features

Adding TimeSeries Features

In [19]:
timeseries_df['TotalAmount'] = timeseries_df['Quantity'] * timeseries_df['Price']

In [20]:
timeseries_df['InvoiceDate'] = pd.to_datetime(timeseries_df['InvoiceDate'])
timeseries_df['Year'] = timeseries_df['InvoiceDate'].dt.year
timeseries_df['Month'] = timeseries_df['InvoiceDate'].dt.month
timeseries_df['DayOfWeek'] = timeseries_df['InvoiceDate'].dt.dayofweek
timeseries_df['HourOfDay'] = timeseries_df['InvoiceDate'].dt.hour
timeseries_df['TimeOfDay'] = pd.cut(timeseries_df['HourOfDay'],
                         bins=[0, 6, 12, 18, 24],
                         labels=['Night', 'Morning', 'Afternoon', 'Evening'])

In [21]:
timeseries_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalAmount,Year,Month,DayOfWeek,HourOfDay,TimeOfDay
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009,12,1,7,Morning
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,1,7,Morning
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009,12,1,7,Morning
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,2009,12,1,7,Morning
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,2009,12,1,7,Morning


Adding Customer Metrics Features

In [22]:
customer_metrics_df['TotalAmount'] = customer_metrics_df['Price'] * customer_metrics_df['Quantity']

In [23]:
customer_metrics_df = customer_metrics_df.groupby(by = 'Customer ID', as_index=False) \
  .agg(
    TotalSpent = ('TotalAmount', 'sum'),
    OrderCount = ('Invoice', 'nunique'),
    FirstInvoiceDate = ('InvoiceDate', 'min'),
    LastInvoiceDate = ('InvoiceDate', 'max')
  )

In [24]:
customer_metrics_df['LastInvoiceDate'] = pd.to_datetime(customer_metrics_df['LastInvoiceDate'])
customer_metrics_df['FirstInvoiceDate'] = pd.to_datetime(customer_metrics_df['FirstInvoiceDate'])

customer_metrics_df['CustomerLifespan'] = (customer_metrics_df['LastInvoiceDate'] - customer_metrics_df['FirstInvoiceDate']).dt.days

In [25]:
#using the maxinvoice date from the table instead of current date since the data is old
max_invoice_date = customer_metrics_df['LastInvoiceDate'].max()

customer_metrics_df['Recency'] = (max_invoice_date - customer_metrics_df['LastInvoiceDate']).dt.days

In [26]:
customer_metrics_df['CustomerLifespan'] = np.where(customer_metrics_df['CustomerLifespan'] == 0, \
                                                1, customer_metrics_df['CustomerLifespan'])

customer_metrics_df['PurchaseFrequency'] = (customer_metrics_df['OrderCount'] / (customer_metrics_df['CustomerLifespan'] / 30)
).round(2)

In [27]:
customer_metrics_df.head()

,Customer ID,TotalSpent,OrderCount,FirstInvoiceDate,LastInvoiceDate,CustomerLifespan,Recency,PurchaseFrequency
0,12346.0,169.36,2,2010-03-02 13:08:00,2010-06-28 13:53:00,118,164,0.51
1,12347.0,1323.32,2,2010-10-31 14:20:00,2010-12-07 14:57:00,37,2,1.62
2,12348.0,221.16,1,2010-09-27 14:59:00,2010-09-27 14:59:00,1,73,30.00
3,12349.0,2221.14,2,2010-04-29 13:20:00,2010-10-28 08:23:00,181,42,0.33
4,12351.0,300.93,1,2010-11-29 15:23:00,2010-11-29 15:23:00,1,10,30.00


### Exporting Data

In [28]:
customer_metrics_df.to_csv('../data/processed/customer_metrics.csv', index=False)
timeseries_df.to_csv('../data/processed/timeseries_data.csv', index=False)